In [1]:
# imports 
import os
import time
import datetime
import json
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn import metrics

from itertools import product


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA

In [2]:
dataDirectory = 'data'

In [3]:
#train = pd.read_csv(dataDirectory+'/train.csv')
structures = pd.read_csv(dataDirectory+'/structures.csv')
#train.shape

In [4]:
structures.head(3)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277


In [5]:
## One hot encode the atom type
structures = pd.get_dummies(structures, columns=['atom'])
structures.head(3)

,molecule_name,atom_index,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0
1,dsgdb9nsd_000001,1,0.002150,-0.006031,0.001976,0,0,1,0,0
2,dsgdb9nsd_000001,2,1.011731,1.463751,0.000277,0,0,1,0,0


In [6]:
## Merge with itself to create atom pairs
structureMerge = pd.merge(structures, structures, how = 'left',left_on  = ['molecule_name'],
                  right_on = ['molecule_name'])
structureMerge.head(3)

,molecule_name,atom_index_x,x_x,y_x,z_x,atom_C_x,atom_F_x,atom_H_x,atom_N_x,atom_O_x,atom_index_y,x_y,y_y,z_y,atom_C_y,atom_F_y,atom_H_y,atom_N_y,atom_O_y
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0,0,-0.012698,1.085804,0.008001,1,0,0,0,0
1,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0,1,0.002150,-0.006031,0.001976,0,0,1,0,0
2,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0,2,1.011731,1.463751,0.000277,0,0,1,0,0


In [7]:
structureMerge['EucDist'] = np.power((structureMerge.x_x-structureMerge.x_y)**2 
                             +   (structureMerge.y_x-structureMerge.y_y)**2 
                             +   (structureMerge.z_x-structureMerge.z_y)**2, 0.5)

In [8]:
## Scale the atom feature by euclidean distance before local aggregation
cols = ['atom_C_y','atom_F_y','atom_H_y','atom_N_y','atom_O_y']
for c in cols:
    structureMerge[c] = structureMerge[c]/(1+structureMerge.EucDist)

In [9]:
## First order spatial graph convolution
firsOrder = structureMerge[['molecule_name','atom_index_x']+cols].groupby(['molecule_name','atom_index_x']).sum().reset_index()

In [10]:
firsOrder.head(3)

,molecule_name,atom_index_x,atom_C_y,atom_F_y,atom_H_y,atom_N_y,atom_O_y
0,dsgdb9nsd_000001,0,1.000000,0.0,1.912092,0.0,0.0
1,dsgdb9nsd_000001,1,0.478022,0.0,2.077919,0.0,0.0
2,dsgdb9nsd_000001,2,0.478023,0.0,2.077918,0.0,0.0


In [12]:
firsOrder.columns = ['molecule_name', 'atom_index', 'atom_C_1', 'atom_F_1', 'atom_H_1',
       'atom_N_1', 'atom_O_1']

In [13]:
## Merge first order features with structures
structures = pd.merge(structures, firsOrder, how = 'left',left_on  = ['molecule_name', 'atom_index'],
                  right_on = ['molecule_name','atom_index'])
structures.head(3)

,molecule_name,atom_index,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O,atom_C_1,atom_F_1,atom_H_1,atom_N_1,atom_O_1
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0,1.000000,0.0,1.912092,0.0,0.0
1,dsgdb9nsd_000001,1,0.002150,-0.006031,0.001976,0,0,1,0,0,0.478022,0.0,2.077919,0.0,0.0
2,dsgdb9nsd_000001,2,1.011731,1.463751,0.000277,0,0,1,0,0,0.478023,0.0,2.077918,0.0,0.0


In [14]:
cols = ['molecule_name', 'atom_index', 'x', 'y', 'z', 'atom_C_1', 'atom_F_1', 'atom_H_1',
       'atom_N_1', 'atom_O_1']

In [15]:
## merge firs order features with itsef to create atom pairs
structureMerge = pd.merge(structures[cols], structures[cols], how = 'left',left_on  = ['molecule_name'],
                  right_on = ['molecule_name'])
structureMerge.head(3)

,molecule_name,atom_index_x,x_x,y_x,z_x,atom_C_1_x,atom_F_1_x,atom_H_1_x,atom_N_1_x,atom_O_1_x,atom_index_y,x_y,y_y,z_y,atom_C_1_y,atom_F_1_y,atom_H_1_y,atom_N_1_y,atom_O_1_y
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1.0,0.0,1.912092,0.0,0.0,0,-0.012698,1.085804,0.008001,1.000000,0.0,1.912092,0.0,0.0
1,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1.0,0.0,1.912092,0.0,0.0,1,0.002150,-0.006031,0.001976,0.478022,0.0,2.077919,0.0,0.0
2,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1.0,0.0,1.912092,0.0,0.0,2,1.011731,1.463751,0.000277,0.478023,0.0,2.077918,0.0,0.0


In [16]:
structureMerge['EucDist'] = np.power((structureMerge.x_x-structureMerge.x_y)**2 
                             +   (structureMerge.y_x-structureMerge.y_y)**2 
                             +   (structureMerge.z_x-structureMerge.z_y)**2, 0.5)

In [17]:
# scale by euclidean distance
cols = ['atom_C_1_y','atom_F_1_y','atom_H_1_y','atom_N_1_y','atom_O_1_y']
for c in cols:
    structureMerge[c] = structureMerge[c]/(1+structureMerge.EucDist)

In [18]:
## generate second order GCN atome feature representations
secondOrder = structureMerge[['molecule_name','atom_index_x']+cols].groupby(['molecule_name','atom_index_x']).sum().reset_index()

In [19]:
secondOrder.head(3)

,molecule_name,atom_index_x,atom_C_1_y,atom_F_1_y,atom_H_1_y,atom_N_1_y,atom_O_1_y
0,dsgdb9nsd_000001,0,1.914024,0.0,5.885260,0.0,0.0
1,dsgdb9nsd_000001,1,1.471315,0.0,5.231766,0.0,0.0
2,dsgdb9nsd_000001,2,1.471315,0.0,5.231766,0.0,0.0


In [20]:
secondOrder.columns = ['molecule_name', 'atom_index', 'atom_C_2', 'atom_F_2', 'atom_H_2',
       'atom_N_2', 'atom_O_2']

In [21]:
structures = pd.merge(structures, secondOrder, how = 'left',left_on  = ['molecule_name', 'atom_index'],
                  right_on = ['molecule_name','atom_index'])
structures.head(3)

,molecule_name,atom_index,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O,atom_C_1,atom_F_1,atom_H_1,atom_N_1,atom_O_1,atom_C_2,atom_F_2,atom_H_2,atom_N_2,atom_O_2
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0,1.000000,0.0,1.912092,0.0,0.0,1.914024,0.0,5.885260,0.0,0.0
1,dsgdb9nsd_000001,1,0.002150,-0.006031,0.001976,0,0,1,0,0,0.478022,0.0,2.077919,0.0,0.0,1.471315,0.0,5.231766,0.0,0.0
2,dsgdb9nsd_000001,2,1.011731,1.463751,0.000277,0,0,1,0,0,0.478023,0.0,2.077918,0.0,0.0,1.471315,0.0,5.231766,0.0,0.0


In [27]:
structures.to_csv('structuresGcn.csv',index=False)

In [22]:
# Sampling the 10% dataframe for quick experiments. This will compromise accuracy !
train = pd.read_csv(dataDirectory+'/train.csv')
train = train.sample(frac=0.1, replace=False, random_state=2019)
train.shape

(465815, 6)

In [23]:
trainMerged = pd.merge(train, structures, how = 'left',left_on  = ['molecule_name', 'atom_index_0'],
                  right_on = ['molecule_name',  'atom_index'])
trainMerged.rename(columns={'atom': 'atom_0','x': 'x_0','y': 'y_0','z': 'z_0'}, inplace=True)


trainMerged = pd.merge(trainMerged, structures, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_1'],
                  right_on = ['molecule_name',  'atom_index'])
trainMerged.rename(columns={'atom': 'atom_1','x': 'x_1','y': 'y_1','z': 'z_1'}, inplace=True)

trainMerged.drop(['atom_index_x','atom_index_y'], axis=1, inplace=True)

trainMerged.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x_0,y_0,z_0,atom_C_x,...,atom_C_1_y,atom_F_1_y,atom_H_1_y,atom_N_1_y,atom_O_1_y,atom_C_2_y,atom_F_2_y,atom_H_2_y,atom_N_2_y,atom_O_2_y
0,1164923,dsgdb9nsd_039011,9,8,2JHC,1.27218,-0.583182,1.971750,0.803563,0,...,2.570713,0.0,2.023518,0.912796,0.0,11.671693,0.0,11.284706,5.339873,0.0
1,4441950,dsgdb9nsd_122989,15,16,3JHH,3.53842,-1.706262,-0.254513,-1.102151,0,...,2.479108,0.0,4.947156,0.000000,0.0,18.883925,0.0,34.042214,0.000000,0.0
2,4566146,dsgdb9nsd_127768,13,4,3JHC,7.92151,0.120346,-2.510468,3.880919,0,...,2.355623,0.0,1.361693,1.360091,0.0,9.234402,0.0,8.302569,6.913899,0.0


In [24]:
def EuclideanDistance(x,y,z):
    """calculates euclidean distance given abs relative position"""
    return np.power(x**2 + y**2 + z**2,0.5)
    
vecDist = np.vectorize(EuclideanDistance)

In [25]:
trainMerged['RelPos_x'] = np.abs(trainMerged['x_0'] - trainMerged['x_1'])
trainMerged['RelPos_y'] = np.abs(trainMerged['y_0'] - trainMerged['y_1'])
trainMerged['RelPos_z'] = np.abs(trainMerged['z_0'] - trainMerged['z_1'])
trainMerged['Euc_Dist'] = vecDist(trainMerged.RelPos_x, trainMerged.RelPos_y ,trainMerged.RelPos_z)

In [26]:
trainMerged.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x_0,y_0,z_0,atom_C_x,...,atom_O_1_y,atom_C_2_y,atom_F_2_y,atom_H_2_y,atom_N_2_y,atom_O_2_y,RelPos_x,RelPos_y,RelPos_z,Euc_Dist
0,1164923,dsgdb9nsd_039011,9,8,2JHC,1.27218,-0.583182,1.971750,0.803563,0,...,0.0,11.671693,0.0,11.284706,5.339873,0.0,0.566169,1.957179,0.778454,2.181075
1,4441950,dsgdb9nsd_122989,15,16,3JHH,3.53842,-1.706262,-0.254513,-1.102151,0,...,0.0,18.883925,0.0,34.042214,0.000000,0.0,1.069689,2.221549,0.489319,2.513752
2,4566146,dsgdb9nsd_127768,13,4,3JHC,7.92151,0.120346,-2.510468,3.880919,0,...,0.0,9.234402,0.0,8.302569,6.913899,0.0,0.103162,0.390069,3.273954,3.298722


In [27]:
labelencoder = LabelEncoder()
def labelEncodeCategoricalFeatures(DF):
    """label encodes the categorical feautes in a given dataframe"""
    df = DF.copy()
    for c in df.columns:
        if df[c].dtype.name == 'object':
            df[c] = labelencoder.fit_transform(df[c])
    return df

In [28]:
## Predictor columns
cols = [c for c in trainMerged.columns if c not in ['id','molecule_name','atom_index_0','atom_index_1','scalar_coupling_constant']]
len(cols)

41

In [29]:
X = trainMerged[cols]
Y = trainMerged['scalar_coupling_constant']

In [30]:
X = labelEncodeCategoricalFeatures(X).values
X.shape

(465815, 41)

In [31]:
## test train split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=2019)
print("Train Test Split:")
print(np.array(X_train).shape, np.array(X_test).shape, np.array(y_train).shape, np.array(y_test).shape)

Train Test Split:
(312096, 41) (153719, 41) (312096,) (153719,)


In [32]:
def competetionMetric(types,y_test, yhat):
    """Metric given here: https://www.kaggle.com/c/champs-scalar-coupling/overview/evaluation"""
    maeList = []
    for t in set(types):
        yt = y_test[types==t]
        yh = yhat[types==t]
        maeList.append(mean_absolute_error(yt, yh))
    return np.mean(np.log(maeList))

In [33]:
def evaluateModel(types,y_test, yhat):
    """Prints several regression evaluation metrics given ground truth and predictions"""
    print('Coefficient of determination: ',r2_score(y_test, yhat))
    print('MAE: ',mean_absolute_error(y_test, yhat))
    print('Competition Metric: ',competetionMetric(types,y_test, yhat))

In [34]:
def naiveBaseLine(X_train, X_test, y_train, y_test):
    """Uses mean of training data as prediction"""
    yhat = np.ones(len(y_test)) * np.mean(y_train)
    types = X_test[:,0]
    evaluateModel(types, y_test, yhat)

In [35]:
def typeAwareBaseLine(X_train, X_test, y_train, y_test):
    """Uses mean of corresponding type as pediction"""
    yhat = np.zeros(len(y_test))
    trainDf = pd.DataFrame({'type':X_train[:,0], 'y':y_train})
    meansDf = trainDf.groupby('type').mean()
    meanDict = dict(zip(meansDf.index,meansDf.y))
    types = X_test[:,0]
    yhat = np.array([meanDict[t] for t in types])
    evaluateModel(types, y_test, yhat)

In [36]:
def randomForestModel(X_train, X_test, y_train, y_test):
    """Trains and evaluates a random forrest regressor"""
    regr = RandomForestRegressor(max_depth=10, random_state=2019,n_estimators=100)
    regr.fit(X_train,y_train)
    yhat = regr.predict(X_test)
    types = X_test[:,0]
    evaluateModel(types,y_test, yhat)

In [37]:
def typeAwareRandomForestModel(X_train, X_test, y_train, y_test):
    """Trains a separate random forrest regressor for each class"""
    models = {}
    types = X_train[:,0]
    for t in set(types):
        yt = y_train[types==t]
        xt = X_train[types==t]
        regr = RandomForestRegressor(max_depth=10, random_state=2019,n_estimators=100)
        regr.fit(xt,yt)
        models[t] = regr
    
    types = X_test[:,0]
    yhat = np.zeros(X_test.shape[0])
    for t in set(types):
        yhat[types==t] = models[t].predict(X_test[types==t])

    evaluateModel(types,y_test, yhat)

In [38]:
naiveBaseLine(X_train, X_test, y_train, y_test)

Coefficient of determination:  -8.45019949569e-08
MAE:  24.7044899378
Competition Metric:  3.00509813563


In [39]:
typeAwareBaseLine(X_train, X_test, y_train, y_test)

Coefficient of determination:  0.949197128083
MAE:  4.22328689846
Competition Metric:  1.23701125805


In [40]:
randomForestModel(X_train, X_test, y_train, y_test)

Coefficient of determination:  0.990594668139
MAE:  2.17728697914
Competition Metric:  0.590545260659


In [41]:
typeAwareRandomForestModel(X_train, X_test, y_train, y_test)

Coefficient of determination:  0.992180651323
MAE:  1.93476493197
Competition Metric:  0.439941656362
